# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [160]:
import pandas as pd
import datetime
import psycopg2

connection = psycopg2.connect(
    host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
    database="mid_term_project",
    user="lhl_student",
    password="lhl_student",
    port=5432
)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [238]:
df_flights = pd.read_sql_query('SELECT fl_date, mkt_carrier, origin, dest, crs_dep_time, crs_arr_time, crs_elapsed_time, arr_delay FROM flights ORDER BY RANDOM() LIMIT 60000;', connection)
df_flights.head()

fl_date mkt_carrier origin dest  crs_dep_time  crs_arr_time  \
0  2018-09-28          DL    ROC  LGA           810           932   
1  2018-06-11          WN    LIT  LAS          1535          1650   
2  2019-02-15          DL    DTW  DSM          2010          2122   
3  2018-01-06          UA    MSP  SFO           600           835   
4  2019-12-22          AS    SFO  IAD           955          1801   

   crs_elapsed_time  arr_delay  
0              82.0       77.0  
1             195.0      -12.0  
2             132.0      -32.0  
3             275.0       12.0  
4             306.0       40.0

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
##### IE statistics based off of other columns: mean delay for january is x. mean delay for morning flights is y
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [239]:
df_features = df_flights.copy()[['mkt_carrier','origin','dest','arr_delay']]
df_features['month'] = df_flights['fl_date'].apply(lambda x: int(x.split('-')[1]))
df_features['day_of_week'] = df_flights['fl_date'].apply(get_day_of_week)
df_features['time_of_day_dep'] = df_flights['crs_dep_time'].apply(lambda x: x//100)
df_features['time_of_day_arr'] = df_flights['crs_arr_time'].apply(lambda x: x//100)
df_features['flight_length'] = df_flights['crs_elapsed_time'].apply(get_flight_length)
df_features = pd.merge(df_features,df_traffic, on=['origin','dest','month'])
df_features.head(20)

mkt_carrier origin dest  arr_delay  month  day_of_week  time_of_day_dep  \
0           DL    ROC  LGA       77.0      9            4                8   
1           DL    DTW  DSM      -32.0      2            4               20   
2           DL    DTW  DSM        NaN      2            4               15   
3           UA    MSP  SFO       12.0      1            5                6   
4           UA    MSP  SFO       12.0      1            5                6   
5           DL    MSP  SFO      -65.0      1            6               11   
6           DL    MSP  SFO      -65.0      1            6               11   
7           UA    MSP  SFO      -24.0      1            6                8   
8           UA    MSP  SFO      -24.0      1            6                8   
9           DL    MSP  SFO      -46.0      1            4               11   
10          DL    MSP  SFO      -46.0      1            4               11   
11          AS    SFO  IAD       40.0     12            6                9   
12          UA    SFO  IAD      -10.0     12            4               10   
13          AA    CVG  LGA      -16.0     10            4                7   
14          AA    CVG  LGA       63.0     10            3               19   
15          DL    CVG  LGA       -7.0     10            6               16   
16          UA    CVG  EWR      -16.0     10            1               20   
17          DL    CVG  EWR      -20.0     10            0               17   
18          DL    CVG  EWR      -20.0     10            0                7   
19          UA    CVG  EWR        8.0     10            1               11   

    time_of_day_arr flight_length  passengers  
0                 9         short        53.0  
1                21         short      4839.0  
2                16         short      4839.0  
3                 8        medium       339.0  
4                 8        medium     11223.0  
5                14        medium       339.0  
6                14        medium     11223.0  
7                11        medium       339.0  
8                11        medium     11223.0  
9                14        medium       339.0  
10               14        medium     11223.0  
11               18        medium       758.0  
12               18        medium       758.0  
13                9         short      3425.0  
14               21         short      3425.0  
15               18         short      3425.0  
16               22         short      2376.0  
17               18         short      2376.0  
18                9         short      2376.0  
19               12         short      2376.0

In [175]:
def get_day_of_week(x):
    a = int(x.split('-')[0])
    b = int(x.split('-')[1])
    c = int(x.split('-')[2])
    dt = datetime.datetime(a,b,c)
    return dt.weekday()

In [194]:
def get_flight_length(x):
    if x < 180:
        return 'short'
    elif x > 360:
        return 'long'
    else:
        return 'medium'

In [206]:
### Airport traffic
df_traffic = pd.read_sql_query('SELECT origin, dest, passengers, month FROM passengers ORDER BY RANDOM() LIMIT 60000;', connection)
df_traffic.head()

origin dest  passengers  month
0        LHR  MSY      1176.0      2
1        BOS  AUG       314.0      3
2        MEM  PDX         0.0      3
3        MSP  MSN        70.0      5
4        SKK  KKA        12.0      6
...      ...  ...         ...    ...
59995    DEN  BUR         0.0     12
59996    FLL  MBJ      3176.0      9
59997    RDV  SRV         0.0      6
59998    LAX  TUS      2406.0      3
59999    AGS  CLT      2014.0      2

[60000 rows x 4 columns]

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.